    Warning: graphviz must be installed on the system, like `sudo pacman -S graphviz`

In [ ]:
%%bash
pip install -Uqq fastai
pip install -Uqq kaggle

In [ ]:
from fastai import *
#from kaggle import api
from pandas.api.types import (
    is_string_dtype,
    is_numeric_dtype,
    is_categorical_dtype)
from fastai.tabular.all import *

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

Hack for fastprogress in vscode (https://github.com/microsoft/vscode-jupyter/issues/1710)

In [ ]:
#from IPython.display import clear_output, DisplayHandle
#def update_patch(self, obj):
#    clear_output(wait=True)
#    self.display(obj)
#DisplayHandle.update = update_patch

# MoA Prediction [Kaggle Competition](https://www.kaggle.com/c/lish-moa/)

## The Dataset

### Download from kaggle

In [ ]:
path = Path("/kaggle/input/lish-moa")

In [ ]:
Path.BASE_PATH = path
#if not path.exists():
#    path.mkdir()
#    api.competition_download_cli('lish-moa', path=path)
#    file_extract(path/'lish-moa.zip')

path.ls(file_type='text')

### Look at the Data

In [ ]:
df = pd.read_csv(path/'train_features.csv', low_memory=False)

In [ ]:
df.columns

In [ ]:
df.nunique()

In [ ]:
df.dtypes

In [ ]:
df_targets_scored = pd.read_csv(path/'train_targets_scored.csv', low_memory=False)

In [ ]:
df_targets_scored.columns

In [ ]:
df_targets_scored['5-alpha_reductase_inhibitor'].unique()

In [ ]:
df_targets_scored.nunique()

In [ ]:
df_targets_scored.dtypes

In [ ]:
dep_vars = df_targets_scored.columns[1:]
dep_vars.size

In [ ]:
df_targets_scored[dep_vars] = df_targets_scored[dep_vars].apply(lambda v: v.astype('float'))
df_targets_scored.dtypes

In [ ]:
df = df.merge(df_targets_scored)

In [ ]:
cont,cat = cont_cat_split(df, 9000, dep_var=[i for i in dep_vars])
cat.remove('sig_id')
len(cont), len(cat)

In [ ]:
cat

In [ ]:
splits = RandomSplitter(valid_pct=0.1)(range_of(df))

In [ ]:
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(df, procs_nn, cat, cont, splits=splits, y_names=[i for i in dep_vars])

In [ ]:
len(to_nn.train.xs), len(to_nn.valid.xs)

In [ ]:
dls = to_nn.dataloaders(1024)

In [ ]:
dls.loss_func

## Learn

In [ ]:
loss = nn.BCEWithLogitsLoss()

In [ ]:
# adapted from https://www.kaggle.com/gunesevitan/mechanisms-of-action-moa-prediction-eda
def mean_columnwise_metric(y_pred, y_true):        
    y_pred = torch.clamp(torch.sigmoid(y_pred.to(dtype=torch.double)), 1e-15, (1 - 1e-15)) 

    score = - torch.mean(torch.mean(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred)))

    return score

In [ ]:
learn = tabular_learner(dls, layers=[500,250], n_out=len(dep_vars), loss_func=loss, metrics=[mean_columnwise_metric])

In [ ]:
lr_min, lr_steep = learn.lr_find()
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learn.fit_one_cycle(14, lr_min, wd=0.4)

In [ ]:
learn.recorder.plot_loss()

## Build the submission

In [ ]:
learn.export('/kaggle/working/models/model.pkl')

In [ ]:
test_df = pd.read_csv(path/'test_features.csv', low_memory=False)
test_df.shape

In [ ]:
test_dl = dls.test_dl(test_df)

In [ ]:
test_preds_raw, _ = learn.get_preds(dl = test_dl)
test_preds_raw.shape

In [ ]:
test_preds = torch.clamp(torch.sigmoid(test_preds_raw.to(dtype=torch.double)), 1e-15, (1-(1e-15)))

In [ ]:
pred_test_df = pd.DataFrame(test_preds)
pred_test_df.insert(0, 'sig_id', test_df['sig_id'])
pred_test_df.columns=df_targets_scored.columns
pred_test_df.head()

In [ ]:
pred_test_df.to_csv("/kaggle/working/submission.csv", index=False)